In [ ]:
import sqlite3

conn = sqlite3.connect("Data/users.db")  # in-memory DB
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    username TEXT,
    password TEXT
)
""")

cursor.execute("INSERT INTO users VALUES (1, 'admin', 'admin123')")
cursor.execute("INSERT INTO users VALUES (2, 'user', 'user123')")

conn.commit()

SQL Injection

In [ ]:
def insecure_login(username):
    query = f"SELECT * FROM users WHERE username = '{username}'"
    print("Executing:", query)
    cursor.execute(query)
    return cursor.fetchall()

# Attacker input
result = insecure_login("' OR 1=1 --")
result

Parameterized Queries

In [ ]:
def secure_login(username):
    query = "SELECT * FROM users WHERE username = ?"
    cursor.execute(query, (username,))
    return cursor.fetchall()

secure_login("' OR 1=1 --")

FILE HANDLING

In [ ]:
# Directory Traversal Example
def read_file(filename):
    with open(filename, "r") as f:
        return f.read()

read_file("../../etc/passwd") 


In [ ]:
#safe file handling
import os

BASE_DIR = "uploads"

def secure_read_file(filename):
    safe_name = os.path.basename(filename)
    full_path = os.path.join(BASE_DIR, safe_name)

    if not full_path.startswith(BASE_DIR):
        raise Exception("Invalid file path")

    return full_path